In [20]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf

x = tf.placeholder("float", shape=[None, 64])
y_ = tf.placeholder("float", shape=[None, 3])

x_image = tf.reshape(x, [-1, 8, 8, 1])


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([4, 4, 1, 12])
b_conv1 = bias_variable([12])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([4, 4, 12, 24])
b_conv2 = bias_variable([24])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([2 * 2 * 24, 1056])
b_fc1 = bias_variable([1056])

h_pool2_flat = tf.reshape(h_pool2, [-1, 2 * 2 * 24])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1056, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()

sess.run(tf.initialize_all_variables())

df_train = pd.read_csv("data/df_train_feature.csv", header=None, sep=',')
df_train_label = pd.read_csv("data/df_train_label.csv", header=None, sep=',')
batchsize = 100

for i in range(1000):
    bids = []
    for ii in range(batchsize):
        batchid = random.randrange(1, len(df_train.index) + 1)
        bids.append(batchid)
    batch_x  = df_train.loc[bids]       
    batch_y_ = df_train_label.loc[bids]  

    if i % 10 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y_, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch_x, y_: batch_y_, keep_prob: 0.5})

